# Load data

In [9]:
import pandas as pd

In [57]:
#activities
residual_800 = pd.read_csv('/Users/esmondchu/Dropbox/UC_Davis/STAT/STA160Spring18/160-Stackoverflow-Data/residuals/data/residuals_800_q.csv')

In [58]:
#questions
x_train_800 = pd.read_csv('/Users/esmondchu/Dropbox/UC_Davis/STAT/STA160Spring18/160-Stackoverflow-Data/residuals/data/X_train_800.csv')

In [60]:
residual_800.head()

,activity,q_num,rank,score,userid
0,answer,0,1,1.234562,571407
1,comment,0,3952,0.120353,1381767
2,comment,0,1,1.234562,571407
3,comment,0,2840,0.206159,1292401
4,comment,0,2821,0.208532,1025201


# FInd outliers

In [82]:
import numpy as np

In [83]:
def outliers(li):
    qua1, qua3 = np.percentile(li, [25, 75])
    iqr = qua3 - qua1
    upper_bound = qua3 + (iqr * 1.5)
    li_outlier = [i for i in li if i >= upper_bound]
    return li_outlier

In [87]:
outlier_rank = outliers(list(residual_800['rank']))

In [114]:
#dataframe with outliers
activities_outliers = residual_800[residual_800['rank'].isin(outlier_rank)]

# Proportion of each activities in outliers

In [138]:
import matplotlib.pyplot as plt

In [139]:
activities_outliers.activity.value_counts()

answer      219
favorite     97
edit         43
comment      30
Name: activity, dtype: int64

# Questions of these observed outlier ranks

In [127]:
outlier_question = x_train_800[x_train_800.index.isin(activities_outliers.q_num.unique())]

In [133]:
outlier_question.head()

,Unnamed: 0,Unnamed: 0.1,Id,ParentId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
2,2,67737,11209935,NaN,NaN,2012-06-26T14:46:59.163,2,10504.0,<p>I 'm getting the date in the following form...,964819.0,NaN,904049.0,Changing one date format to another,java date format,4.0,1,NaN,NaN
13,13,1296,8741054,NaN,8741143.0,2012-01-05T10:37:18.253,0,1732.0,<p>How to resolve warning message coming in ec...,374199.0,NaN,1977903.0,Eclipse Warning message,java eclipse,5.0,6,0.0,NaN
14,14,43834,10314404,NaN,10314451.0,2012-04-25T11:09:04.190,-2,2842.0,<p>I am working on Galaxy Nexus(Android 4.0.3)...,896579.0,NaN,136445.0,How to save captured image to gallery,java android emulation samsung-mobile,2.0,1,NaN,NaN
23,23,68460,11239086,NaN,11239128.0,2012-06-28T06:23:16.197,132,291504.0,<blockquote>\r\r\r\n <p><strong>Possible Dupl...,1439243.0,NaN,-1.0,java.lang.UnsupportedClassVersionError Unsuppo...,java,7.0,0,33.0,2013-02-04T07:29:18.850
24,24,20678,9484820,NaN,NaN,2012-02-28T15:33:14.167,-1,1488.0,<p>I have an html file called basic.html and w...,1178770.0,NaN,1400768.0,Regex - find various strings from an HTML file,java html regex,2.0,4,NaN,2014-09-27T14:44:12.220


In [163]:
y_train = pd.read_csv('/Users/esmondchu/Dropbox/UC_Davis/STAT/STA160Spring18/160-Stackoverflow-Data/residuals/data/y_train.csv')

# map activity for feature matrix (question number)

In [331]:
def map_activity(file_name):
    """ Modify Feature Matrix (Ranks of users for each question) to add activities.
    
    Argument: file_name (int) - the question number
    
    Return: pd.dataframe - the modified feature matrix
    """
    
    path = '/Users/esmondchu/Dropbox/UC_Davis/STAT/STA160Spring18/160-Stackoverflow-Data/residuals/feature_matrices/'\
    + 'q_' + str(file_name) + '_feature_matrix.csv'
    
    que = pd.read_csv(path, header = None)
    user_activity = residual_800[residual_800.q_num == file_name]
    user_activity_dict = {i:j for i, j in zip(user_activity.userid, user_activity.activity)}

    activity_que = []
    for i in que[0]:
        if i in user_activity_dict.keys():
            activity_que.append(user_activity_dict.get(i))
        else:
            activity_que.append('no_act')
    que['activity'] = activity_que
    
    que.sort_values(4)
    que['rank'] = que.index
    return que

In [525]:
def get_feature_rank(num):
    """ Use function: map_activity to run
    
    Argument: num (int) - question number (feature matrid file)
    
    Return: pd.DataFrame - the dataframe with sorted feature rank
    """
    que_no_white = map_activity(num)[map_activity(num).activity != 'no_act']

    q_num = []
    user = []
    feature_rank = []
    activity = []
    rank = []

    for i in que_no_white.index:
        users = que_no_white.loc[i][1:4]
        users_dic = {i:j for i, j in zip(users.index, users.values)}
        sorted_dic = sorted(users_dic, key=users_dic.get)
    
        q_num.append(num)
        user.append(que_no_white.loc[i,0])
        feature_rank.append(sorted_dic)
        activity.append(que_no_white.loc[i]['activity'])
        rank.append(que_no_white.loc[i, 'rank'])
    
    out_dict = {'q_num':q_num, 'user':user, 'feature_rank':feature_rank, 'activity':activity, 'rank':rank}
    out_df = pd.DataFrame(out_dict)
    out_df = out_df[['q_num','user','rank','feature_rank','activity']]
    
    return out_df

In [526]:
#example
get_feature_rank(101)

,q_num,user,rank,feature_rank,activity
0,101,571407.0,2,"[1, 3, 2]",comment
1,101,537445.0,309,"[3, 2, 1]",favorite
2,101,177800.0,1792,"[1, 3, 2]",edit
3,101,422437.0,4452,"[1, 3, 2]",favorite
